In [1]:
import tensorflow as tf
import numpy as np

# tf.enable_eager_execution()
# tf.executing_eagerly()

/Users/youncheol/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
sequence_length = 10
embedding_dim = 100
filter_size = 3
num_of_filters = 50
batch_size = 16
class_num = 6

## Input Vector Matrices
word_vector = np.random.randint(0, 10, size=(batch_size, 10, 100))
inputs = word_vector.astype('float32')

tf.reset_default_graph()

with tf.variable_scope('cnn_attention'):
    input_reshaped = tf.reshape(inputs, [-1, sequence_length, embedding_dim, 1])
    paddings = tf.constant([[0, 0], [1, 1], [0, 0], [0, 0]], dtype='int32')
    input_padded = tf.pad(input_reshaped, paddings, 'CONSTANT')
    
    filter_shape = [filter_size, embedding_dim, 1, num_of_filters]
    
    w = tf.get_variable('cnn_filter', 
                        shape=filter_shape, 
                        initializer=tf.truncated_normal_initializer(stddev=0.1))
    
    b = tf.get_variable('cnn_bias',
                        shape=[num_of_filters],
                        initializer=tf.constant_initializer(0.0))
    
    conv = tf.nn.conv2d(input=input_padded, 
                        filter=w, 
                        strides=[1, 1, 1, 1], 
                        padding='VALID',
                        name='convolution')
    
    c = tf.nn.relu(tf.nn.bias_add(conv, b), name='relu')
    c = tf.reduce_mean(tf.transpose(tf.squeeze(c), perm=[0, 2, 1]), 
                       axis=1, 
                       name='average_across_filter')
    
    
with tf.variable_scope('lstm_unit'):
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(100)
    lstm_cell = tf.nn.rnn_cell.DropoutWrapper(lstm_cell, output_keep_prob=0.5)
    h, _ = tf.nn.dynamic_rnn(lstm_cell, inputs, dtype=tf.float32)
    
    
with tf.variable_scope('make_output'):    
    c = tf.expand_dims(c, axis=2)
    s = tf.reduce_mean(tf.multiply(h, c), axis=1)
    
    w = tf.get_variable('output_weights', 
                        shape=[embedding_dim, class_num], 
                        initializer=tf.truncated_normal_initializer(stddev=0.1))
    
    b = tf.get_variable('output_bias',
                        shape=[class_num],
                        initializer=tf.constant_initializer(0.0))
    
    p = tf.nn.bias_add(tf.matmul(s, w), b)

In [6]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    context = sess.run(c)
    wt = sess.run(W)
    output = sess.run(s)
    pp = sess.run(p)

    print(context.shape)
    print(wt.shape)
    print(output.shape)
    print(p.shape)

(16, 10, 1)
(100, 6)
(16, 100)
(16, 6)
